In [1]:
import pandas as pd
import numpy as np
import gc

In [ ]:
#Load data from excel and split by "|"
df = pd.read_csv('D:/FIM 500 Project/Excel raw data/2008Q1.csv',header = 0,names = ['Reference Pool ID', 'Loan Identifier', 'Monthly Reporting Period', 'Channel', 'Seller Name', 
     'Servicer Name', 'Master Servicer', 'Original Interest Rate', 'Current Interest Rate', 'Original UPB', 
     'UPB at Issuance', 'Current Actual UPB', 'Original Loan Term', 'Origination Date', 'First Payment Date', 
     'Loan Age', 'Remaining Months to Legal Maturity', 'Remaining Months To Maturity', 'Maturity Date', 'Original Loan to Value Ratio (LTV)' 
    ,'Original Combined Loan to Value Ratio (CLTV)', 'Number of Borrowers', 'Debt-To-Income (DTI)', 'Borrower Credit Score at Origination', 
     'Co-Borrower Credit Score at Origination', 'First Time Home Buyer Indicator', 'Loan Purpose', 'Property Type', 'Number of Units', 
     'Occupancy Status', 'Property State', 'Metropolitan Statistical Area (MSA)', 'Zip Code Short', 'Mortgage Insurance Percentage', 
     'Amortization Type', 'Prepayment Penalty Indicator', 'Interest Only Loan Indicator', 'Interest Only First Principal And Interest Payment Date', 
     'Months to Amortization', 'Current Loan Delinquency Status', 'Loan Payment History', 'Modification Flag', 'Mortgage Insurance Cancellation Indicator', 
     'Zero Balance Code', 'Zero Balance Effective Date', 'UPB at the Time of Removal', 'Repurchase Date', 'Scheduled Principal Current', 
     'Total Principal Current', 'Unscheduled Principal Current', 'Last Paid Installment Date', 'Foreclosure Date', 'Disposition Date', 
     'Foreclosure Costs', 'Property Preservation and Repair Costs', 'Asset Recovery Costs', 'Miscellaneous Holding Expenses and Credits', 'Associated Taxes for Holding Property', 
     'Net Sales Proceeds', 'Credit Enhancement Proceeds', 'Repurchase Make Whole Proceeds', 'Other Foreclosure Proceeds', 'Non-Interest Bearing UPB', 'Principal Forgiveness Amount' 
     ,'Original List Start Date', 'Original List Price', 'Current List Start Date', 'Current List Price', 'Borrower Credit Score At Issuance', 
     'Co-Borrower Credit Score At Issuance', 'Borrower Credit Score Current', 'Co-Borrower Credit Score Current', 'Mortgage Insurance Type', 'Servicing Activity Indicator' 
    ,'Current Period Modification Loss Amount', 'Cumulative Modification Loss Amount', 'Current Period Credit Event Net Gain or Loss', 'Cumulative Credit Event Net Gain or Loss', 
     'HomeReady Program Indicator', 'Foreclosure Principal Write-off Amount', 'Relocation Mortgage Indicator', 'Zero Balance Code Change Date', 'Loan Holdback Indicator', 
     'Loan Holdback Effective Date', 'Delinquent Accrued Interest', 'Property Valuation Method', 'High Balance Loan Indicator', 'ARM Initial Fixed-Rate Period ≤ 5 YR Indicator', 
     'ARM Product Type', 'Initial Fixed-Rate Period', 'Interest Rate Adjustment Frequency', 'Next Interest Rate Adjustment Date', 'Next Payment Change Date', 'Index', 
     'ARM Cap Structure', 'Initial Interest Rate Cap Up Percent', 'Periodic Interest Rate Cap Up Percent', 'Lifetime Interest Rate Cap Up Percent', 'Mortgage Margin' 
    ,'ARM Balloon Indicator', 'ARM Plan Number', 'Borrower Assistance Plan', 'High Loan to Value (HLTV) Refinance Option Indicator', 'Deal Name' 
    ,'Repurchase Make Whole Proceeds Flag', 'Alternative Delinquency Resolution', 'Alternative Delinquency Resolution Count', 'Total Deferral Amount'
], delimiter = '|')


In [ ]:
df.head()

In [ ]:
len(df['Loan Identifier'].unique())

In [ ]:
# Main LGD model: Loss = "Loan Balance" + Costs – Property Sales Price – Mortgage Insurance Payment
# Factor related to "Loan Balance" : Original UPB(10), UPB at Issuance(11),Current Actual UPB(12) 
Loan_Balance = {1:'Original UPB', 2:'UPB at Issuance', 3:'Current Actual UPB'}
Cols_Loan_Balance = df[['Original UPB', 'UPB at Issuance', 'Current Actual UPB']]
Cols_Loan_Balance.head()

In [ ]:
# Main model: Loss = Loan Balance + "Costs" – Property Sales Price – Mortgage Insurance Payment
# Factor related to "Costs" : Asset Recovery Costs (56), Miscellaneous Holding Expenses and Credits (57)
# Associated Taxes for Holding Property (58), Foreclosure Costs (54)
# Property Preservation and Repair Costs (55)

Costs = {1:'Asset Recovery Costs', 2:'Miscellaneous Holding Expenses and Credits', 3:'Associated Taxes for Holding Property',
        4:'Foreclosure Costs', 5:'Property Preservation and Repair Costs'}
Cols_costs = df[['Asset Recovery Costs', 'Miscellaneous Holding Expenses and Credits',
                'Associated Taxes for Holding Property', 'Foreclosure Costs', 'Property Preservation and Repair Costs']]
Cols_costs.head()

In [ ]:
# Main model: Loss = Loan Balance + Costs – "Property Sales Price" – Mortgage Insurance Payment
# Factors relate to Property Sales Price(column #): Net Sales Proceeds (59), Other Foreclosure Proceeds (62), Property Type (28)
Property_Sales_Price = {1:'Net Sales Proceeds', 2:'Other Foreclosure Proceeds', 3:'Property Type'}
Cols_Property_Sales_Price = df[['Net Sales Proceeds', 'Other Foreclosure Proceeds', 'Property Type']]
Cols_Property_Sales_Price.head()

In [ ]:
# Main model: Loss = Loan Balance + Costs – Property Sales Price – "Mortgage Insurance Payment"
# Factors relate to Mortgage Insurance Payment(column #): Credit Enhancement Proceeds (60), Mortgage Insurance Percentage (34), 
# Mortgage Insurance Cancellation Indicator (43), Mortgage Insurance Type (73)
Mortgage_Insurance_Payment = {1:'Credit Enhancement Proceeds', 2:'Mortgage Insurance Percentage',
                              3:'Mortgage Insurance Cancellation Indicator', 4:'Mortgage Insurance Type'}
Cols_Mortgage_Insurance_Payment = df[['Credit Enhancement Proceeds', 'Mortgage Insurance Percentage',
                        'Mortgage Insurance Cancellation Indicator', 'Mortgage Insurance Type']]
Cols_Mortgage_Insurance_Payment.head()

In [ ]:
# It will lead to memory error!
Lgddf = Cols_Loan_Balance.append(Cols_costs).append(Cols_Property_Sales_Price).append(Cols_Mortgage_Insurance_Payment)
Lgddf.head()


In [ ]:
del df
gc.collect()

In [ ]:
# Use .describe() to find summary statistics of columns 
SS_Loan_Balance = Cols_Loan_Balance.describe()
SS_costs = Cols_costs.describe()
SS_Property_Sales_Price = Cols_Property_Sales_Price.describe()
SS_Mortgage_Insurance_Payment = Cols_Mortgage_Insurance_Payment.describe()

In [ ]:
SS_Mortgage_Insurance_Payment

In [ ]:
# Use .isna().sum() to find that all data in column "UPB at Issuance" is NaN 
Cols_Loan_Balance.isna().sum()